# Step 4: Rebalnacing Plan 

## 4.1 Processing Data

### 4.1.1 Get the result of bicycles balance

#### Input：

    od_hourly_zip_poisson_daytype_1.csv
       
#### Output：

    station_daily_balance_daytype_1.csv

In [ ]:
import pandas as pd

# Read demand prediction file
df = pd.read_csv("od_hourly_zip_poisson_daytype_1.csv")
demand_col = "demand_zip_poisson"

# Aggregate departures
borrow = (
    df.groupby("start_station_id")[demand_col]
    .sum()
    .rename("borrowed")
)

# Aggregate returns
returned = (
    df.groupby("end_station_id")[demand_col]
    .sum()
    .rename("returned")
)

# Merge into station-level table
station_balance = pd.concat([borrow, returned], axis=1).fillna(0)

# Compute net flow
station_balance["net"] = station_balance["returned"] - station_balance["borrowed"]

# Define move_in / move_out
station_balance["move_out"] = station_balance["net"].apply(lambda x: x if x > 0 else 0)
station_balance["move_in"] = station_balance["net"].apply(lambda x: -x if x < 0 else 0)

# Reset index to obtain station_id column
station_balance = station_balance.reset_index().rename(columns={"index":"station_id"})

# Save results
station_balance.to_csv("station_daily_balance_daytype_1.csv", index=False)

station_balance.head()

### 4.1.2 Combine data for caulating distance

#### Input：

    station_daily_balance_daytype_1.csv 
    merged_2018_2021.csv      
#### Output：

    station_tasks.csv

 Merge station daily net flows, coordinates, and capacity into a single task table.
 
 Read daily net demand, join with station locations and capacity plan, filter invalid coordinates, and export a cleaned per-station dataset for visualization or further analysis.


In [ ]:
import pandas as pd

# Read daily rebalancing results
balance = pd.read_csv("station_daily_balance_daytype_1.csv")
# Keep only the required columns
balance = balance[["station_id", "move_out", "net","move_in"]]


# Extract station coordinates 
df = pd.read_csv("merged_2018_2021.csv")

# Extract start_station_id and its latitude/longitude
loc = df[["start_station_id", "start_station_latitude", "start_station_longitude"]].drop_duplicates()

# Rename columns 
loc = loc.rename(columns={
    "start_station_id": "station_id",
    "start_station_latitude": "latitude",
    "start_station_longitude": "longitude"
})

#  Extract initial stock x_i and capacity y_i from Minlimit_capacity_daytype_1.csv
cap = pd.read_csv("Minlimit_capacity_daytype_1.csv")

cap = cap.rename(columns={
    "x_i": "initial_stock",
    "y_i": "capacity"
})

# Merge the three tables

# Merge balance with location
df_merge = balance.merge(loc, on="station_id", how="left")

# Merge with capacity info
df_merge = df_merge.merge(cap[["station_id", "initial_stock", "capacity"]],
                          on="station_id", how="left")


#Remove stations with missing coordinates

# Latitude/longitude are left-joined from loc; missing ones are NaN
print(f"Station count before dropping: {len(df_merge)}")

# Drop rows with latitude or longitude is NaN
df_merge.dropna(subset=["latitude", "longitude"], inplace=True)

# Convert to numeric types, drop invalid values 
df_merge["latitude"] = pd.to_numeric(df_merge["latitude"], errors='coerce')
df_merge["longitude"] = pd.to_numeric(df_merge["longitude"], errors='coerce')
df_merge.dropna(subset=["latitude", "longitude"], inplace=True)

print(f"Station count after dropping: {len(df_merge)}")




# Remove duplicate stations
df_merge = df_merge.drop_duplicates(subset=["station_id"], keep="first")


# Reorder columns
df_merge = df_merge[[
    "station_id",
    "latitude",
    "longitude",
    "initial_stock",
    "capacity",
    "net",
    "move_out",
    "move_in"
]]

# Export the final station task file
df_merge.to_csv("station_tasks.csv", index=False)

df_merge.head()

## 4.2 Genetic Algorithm

#### Input：

    station_tasks.csv


Genetic algorithm–based vehicle routing for bike rebalancing.

Firstly, builds a distance matrix, selects a heuristic depot, and then runs a DEAP GA with random-key encoding to search for low-cost routes.

The fitness function enforces capacity, vehicle count, and full-demand-service constraints via penalties.

After optimization, the best solution is decoded into vehicle routes and printed.



11.19Record：

We tried using the simulated annealing algorithm to avoid local optima and premature convergence, but it ran too long to get useful results—so we cut this part from the final code.

In [4]:
import pandas as pd
import numpy as np
from deap import base, creator, tools, algorithms
import random
from haversine import haversine, Unit
import sys
import time


print("Part 1: Data preparation and index mapping")


# (1.1) Read full station information from station_task.csv
df_task = pd.read_csv("station_tasks.csv")

# Ensure station_id is string
df_task["station_id"] = df_task["station_id"].astype(str)

# Make sure move_out / move_in / stock / capacity / net to be integers
df_task["move_out"] = df_task["move_out"].astype(int)
df_task["move_in"] = df_task["move_in"].astype(int)
df_task["initial_stock"] = df_task["initial_stock"].astype(int)
df_task["capacity"] = df_task["capacity"].astype(int)
df_task["net"] = df_task["net"].astype(int)


# (1.2)  Build station list

# Keep stations with complete coordinates
df_station_lacation = df_task.dropna(subset=['latitude','longitude']).copy()
df_station_clean = df_station_lacation[df_station_lacation['net'] != 0].copy()
# Rebuild station list
STATIONS = df_station_clean['station_id'].astype(str).tolist()
print(f"Number of stations: {len(STATIONS)}")


# (1.3) Build station_id as integer index mapping
id_to_int = {sid: idx for idx, sid in enumerate(STATIONS)}
int_to_id = {idx: sid for sid, idx in id_to_int.items()}

# Synchronize integer index mapping
id_to_int = {sid: idx for idx, sid in enumerate(STATIONS)}
int_to_id = {idx: sid for sid, idx in id_to_int.items()}
STATIONS_INT = list(id_to_int.values())

print(f"Number of stations with valid coordinates：{len(STATIONS_INT)}")
STATIONS_INT = list(int_to_id.keys())
N_STATIONS = len(STATIONS_INT)
print("Established station_id <-> index mapping")


# (1.4) Build initial_stock dictionary
initial_stock = {
    id_to_int[row.station_id]: row.initial_stock
    for row in df_station_clean.itertuples()
}


# (1.5) Build capacity dictionary
capacity = {
    id_to_int[row.station_id]: row.capacity
    for row in df_station_clean.itertuples()
}


# (1.6) Read demand move_out / move_in / net
demand_U = {id_to_int[row.station_id]: row.move_out for row in df_station_clean.itertuples()}  # demand to upload (U)
demand_D = {id_to_int[row.station_id]: row.move_in for row in df_station_clean.itertuples()}   # demand to download (D)
Net_Demand = {id_to_int[row.station_id]: row.net for row in df_station_clean.itertuples()} 

print(f" total move_in = {sum(demand_U.values())}  total move_out = {sum(demand_D.values())}")


# (1.7) Build coordinate dictionary from latitude/longitude
station_coords_raw = {
    row.station_id: {
        "lat": float(row.latitude),
        "lon": float(row.longitude)
    }
    for row in df_station_clean.itertuples()
}

print("Loaded all station coordinates")


# (1.8) Compute distance matrix over integer indices
distance_matrix = pd.DataFrame(index=STATIONS_INT, columns=STATIONS_INT, dtype=float)
for i in STATIONS_INT:
    for j in STATIONS_INT:
        if i == j:
            distance_matrix.loc[i, j] = 0.0
            continue
        # Decode back to original ID
        sid_i = int_to_id[i] 
        sid_j = int_to_id[j]
        # Lookup coordinates
        coord_i = (station_coords_raw[sid_i]['lat'], station_coords_raw[sid_i]['lon'])
        coord_j = (station_coords_raw[sid_j]['lat'], station_coords_raw[sid_j]['lon'])
        # Compute Haversine distance (km)
        distance_matrix.loc[i, j] = haversine(coord_i, coord_j, unit=Unit.KILOMETERS)

# Save distance matrix
output_filename = 'distance_matrix_int_km.csv'
distance_matrix.to_csv(output_filename)
print(f"Distance matrix over integer indices saved to {output_filename}")


# Heuristic computation of best depot candidate
print("Part2: computing heuristic best depot candidate...")
best_depot_int = -1
min_total_distance = float('inf') #positive inf


# （2.1）Iterate over all stations as candidate depot
for candidate_depot_int in STATIONS_INT:
    
    # Total distance from this depot to all others (single-trip sum as ranking indicator)
    total_distance_sum = distance_matrix.loc[candidate_depot_int, :].sum()
    
    if total_distance_sum < min_total_distance:
        min_total_distance = total_distance_sum
        best_depot_int = candidate_depot_int

# （2.2）Update GA candidate depot list to only contain this best station
if best_depot_int != -1:
    # Decode integer index to original ID
    best_depot_id = int_to_id[best_depot_int]
    
    print(f"Heuristic result: best depot candidate is {best_depot_id} (index: {best_depot_int})")
    
    # Save this best integer index for GA loop
    OPTIMAL_CANDIDATE_DEPOTS_INT = [best_depot_int]
else:
    # Fallback in case of unexpected issue
    OPTIMAL_CANDIDATE_DEPOTS_INT = STATIONS_INT[:1]



# （2.4）：Define cost and vehicle parameters ---
Cost_Movement_KM = 0.8      #fixed cost per km per vehicle
Vehicle_Capacity_Dict = {1: 30, 2: 30, 3: 30, 4: 30, 5: 30, 6: 30, 7: 40, 8: 40, 9: 40, 10: 40} # 定义每辆车的容量  K=1, K=2
VEHICLES = list(Vehicle_Capacity_Dict.keys())

print(f"Data preparation finished. {N_STATIONS} valid stations in total.")


# Part3：DEAP framework definition

# （3.1） Define objective: maximize fitness (weight > 0)
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# （3.2） Define Individual based on list (for random-key float chromosome) using FitnessMax
creator.create("Individual", list, fitness=creator.FitnessMax)

# （3.3） Initialize Toolbox
toolbox = base.Toolbox()

# （3.4）Register gene generator (Attribute Generator)
toolbox.register("attr_float", random.random)



# （3.4）Fitness Function

# Big M：
PENALTY_UNSERVED_DEMAND = 1000000.0  # penalty for unserved stations
PENALTY_INVALID_PATH = 100000000.0     # penalty for invalid paths 

# Update fitness function signature to accept MAX_K_ALLOWED
def calculate_vrp_fitness(paths_by_vehicle_int, K, distance_matrix, 
                            Vehicle_Capacity_Dict, demand_U, demand_D, 
                            depot_int, Cost_Movement_KM, MAX_K_ALLOWED): 
    """
     Parameters
    ----------
    paths_by_vehicle_int : dict
        Vehicle routes {vehicle_id: [Depot, 1, 2, ..., Depot]}.
    K : int
        Current maximum number of vehicles considered in cost computation.
    MAX_K_ALLOWED : int
        Hard upper bound on the number of vehicles allowed.
    Other arguments provide distances, capacities, and demand dictionaries.


        Fitness value = 1.0 / total_cost (we maximize fitness).
    """
    

    # Constraint 1: all stations that must be served (non-zero net demand, excluding depot)
    required_stations = set(demand_U.keys()) 
    if depot_int in required_stations:
        required_stations.remove(depot_int)

    cost_vrp_travel = 0.0
    served_stations = set()
    used_vehicles_count = 0 
    

    # （3.4.1）Traverse paths and check load/cost 
    for k_id, path in paths_by_vehicle_int.items(): # k_id is vehicle ID
        
        # Constraint 2: Valid path must have at least Depot -> Station -> Depot
        if len(path) > 2: 
            used_vehicles_count += 1
            
            # Get current vehicle capacity
            vehicle_capacity = Vehicle_Capacity_Dict.get(k_id)
                
            current_load = 0 # vehicle starts from depot with load 0
            
            # Constraint3: must start and end at depot
            if path[0] != depot_int or path[-1] != depot_int:
                return 1.0 / PENALTY_INVALID_PATH 

            # Traverse path
            for i in range(len(path) - 1):
                start_node = path[i]
                end_node = path[i+1]
                
                # Accumulate travel cost
                distance = distance_matrix.loc[start_node, end_node]
                cost_vrp_travel += distance * Cost_Movement_KM
                
                # Update load and check capacity
                if end_node != depot_int:
                    # Mark served station
                    served_stations.add(end_node)
                   
                    # Load change = net demand at this station
                    service_amount = Net_Demand.get(end_node, 0)
                    current_load += service_amount 

                    # Constraint 4 Load must be within [0, capacity]
                    if not ( 0 <= current_load <= vehicle_capacity): # 检查 L >= 0 (不能运送负数货物) 且 L <= C (不能超载)
                        return 1.0 / PENALTY_INVALID_PATH # 载重违规



        elif len(path) == 2 and path[0] == depot_int and path[1] == depot_int:
            # Pure depot-depot path: vehicle not used
            continue


            
    # Vehicle count and demand coverage constraints

    #Objective Function
    total_cost = cost_vrp_travel 

    # Vehicle count penalty (hard constraint via MAX_K_ALLOWED)
    if used_vehicles_count > MAX_K_ALLOWED:
        excess_vehicles = used_vehicles_count - MAX_K_ALLOWED
        total_cost += excess_vehicles * PENALTY_INVALID_PATH 

    # Unserved stations penalty: all required stations must be served
    if not required_stations.issubset(served_stations):
        unserved_count = len(required_stations - served_stations)
        total_cost += unserved_count * PENALTY_UNSERVED_DEMAND
        
    
    # Return fitness
    if total_cost > 0:
        return 1.0 / total_cost   # the lower cost, the higher fitness
    else:
        # Invalid solution, return large fitness
        return 1e9

print("VRP fitness function defined")

#（3.5）GA main loop

# GA parameters for DEAP
ga_parameters = {
    'max_num_iteration': 300,         # number of generations
    'population_size': 1000,           # population size   
    'parents_portion': 0.5,          # parent ratio (implicit in selection)
    'mutation_probability': 0.1,     # mutation probability
    'crossover_probability': 0.9,    # crossover probability
    'elit_ratio': 0.01,              # elite ratio (via HallOfFame)
    'crossover_type': 'uniform',     # crossover type: uniform
    'selection_type': 'roulette',    #  selection type: roulette
    'max_iteration_without_improv': None # avoid premature convergence
}


max_vehicles = 3 #  maximum number of vehicles to test
best_overall_cost = float('inf')
best_solution = {}

start_time = time.time()

# Loop over candidate depots
for depot_int in OPTIMAL_CANDIDATE_DEPOTS_INT:
    
    # Stations to visit (exclude depot)
    stations_to_visit_int = [i for i in STATIONS_INT if i != depot_int]

    # Loop over number of vehicles K
    for k in range(1, max_vehicles + 1):
        
        print(f"--- Running GA: Depot={int_to_id[depot_int]}, K={k} vehichcles ---")
        
        # Gene pool = stations_to_visit + (k-1) depot separators
        genes_to_permute = stations_to_visit_int + [depot_int] * (k - 1)
        dimension = len(genes_to_permute)
        
        if dimension == 0:
            continue

        # Fitness function wrapper for DEAP (Random-Key decoding)
        def fitness_wrapper_deap(chromosome):
            # 1. Decode: sort by gene value to get visiting order
            permutation_indices = np.argsort(chromosome)
            
            # 2. Map to integer station IDs via gene pool
            ordered_path_list = [genes_to_permute[i] for i in permutation_indices]
            
            # 3. Split path into vehicle routes using depot as separator
            paths_by_vehicle_int = {}
            current_path = [depot_int]
            current_k = 1 
            

            for station_id_int in ordered_path_list:
                if station_id_int == depot_int: 

                    # Close current route and start a new one from depot
                    current_path.append(depot_int)
                    paths_by_vehicle_int[current_k] = current_path
                    current_k += 1 
                    current_path = [depot_int] 
                else:
                    current_path.append(station_id_int)
            

            # Close last route
            current_path.append(depot_int)
            paths_by_vehicle_int[current_k] = current_path
            
            # Call fitness function
            fitness_value = calculate_vrp_fitness(
                paths_by_vehicle_int=paths_by_vehicle_int, 
                K=k, 
                distance_matrix=distance_matrix, 
                Vehicle_Capacity_Dict=Vehicle_Capacity_Dict, 
                demand_U=demand_U, 
                demand_D=demand_D, 
                depot_int=depot_int, 
                Cost_Movement_KM=Cost_Movement_KM,
                MAX_K_ALLOWED=k # # pass K as hard upper bound
            )
            return (fitness_value,)
        
        # Unregister old individual/population definitions if present
        if "individual" in toolbox.__dict__: 
            toolbox.unregister("individual")
        if "population" in toolbox.__dict__: 
            toolbox.unregister("population")
            
        # Register Individual and Population generator for current dimension
        toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=dimension)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        
        # Register GA operators
        toolbox.register("evaluate", fitness_wrapper_deap)                   
        toolbox.register("mate", tools.cxUniform, indpb=0.5)                 
        toolbox.register("mutate", tools.mutGaussian, mu=0.0, sigma=0.1, 
                         indpb=ga_parameters['mutation_probability'])       
        toolbox.register("select", tools.selRoulette)                        

        # Initialize population, hall of fame, and stats
        pop = toolbox.population(n=ga_parameters['population_size'])
        hof = tools.HallOfFame(1) 
        
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("avg", np.mean)
        stats.register("min", np.min)
        stats.register("max", np.max)


        # Run GA (eaSimple algorithm)
        pop, log = algorithms.eaSimple(pop, toolbox, 
                                       cxpb=ga_parameters['crossover_probability'], # 交叉概率
                                       mutpb=ga_parameters['mutation_probability'], # 变异概率
                                       ngen=ga_parameters['max_num_iteration'],    # 迭代次数
                                       stats=stats, 
                                       halloffame=hof, 
                                       verbose=False) 

        # Extract best result
        if hof and hof[0].fitness.values[0] > 0:
            best_chromosome = hof[0] 
            best_function = hof[0].fitness.values[0] 
            current_cost = 1 / best_function

        else:
            current_cost = float('inf')
        
        print(f"    K={k} finished. minmum cost: {current_cost:.2f}")

        # Record best overall solution
        if current_cost < best_overall_cost:
            best_overall_cost = current_cost
            best_solution = {
                'Depot_Int': depot_int,
                'K_Vehicles': k,
                'Total_Cost': current_cost,
                'Optimal_Path_Chromosome (Float-Key)': np.array(best_chromosome) 
            }
        

# Part 4: Decode and output final result
end_time = time.time()
print(f"\n Optimization finished (total time: {end_time - start_time:.2f} s) ")

if not best_solution:
    print("No feasible solution found.")
else:
    print(f"Depot: {int_to_id.get(best_solution.get('Depot_Int'), 'N/A')}")
    print(f"Vehicles: {best_solution.get('K_Vehicles')}")
    print(f"Minimum total cost: {best_solution.get('Total_Cost'):.2f}")
    
    # Decode final path
    final_chromosome = best_solution.get('Optimal_Path_Chromosome (Float-Key)')
    depot_int = best_solution.get('Depot_Int')
    k = best_solution.get('K_Vehicles')
    
    if final_chromosome is not None and depot_int is not None:
        stations_to_visit_int = [i for i in STATIONS_INT if i != depot_int]
        # Rebuild gene pool for correct decoding
        genes_to_permute = stations_to_visit_int + [depot_int] * (k - 1)
        
        permutation_indices = np.argsort(final_chromosome)
        ordered_path_list = [genes_to_permute[i] for i in permutation_indices]

        final_paths = {}
        current_path = [depot_int]
        current_k = 1
        # Split path based on depot separators
        for station_id_int in ordered_path_list:
            if station_id_int == depot_int:
                current_path.append(depot_int)
                final_paths[current_k] = current_path
                current_k += 1
                current_path = [depot_int]
            else:
                current_path.append(station_id_int)
        current_path.append(depot_int)
        final_paths[current_k] = current_path
        
        print("\n Final rebalancing routes")
        for k_id, path_int in final_paths.items():
            if len(path_int) > 2:
                path_str = [int_to_id[i] for i in path_int]
                print(f"Vehicles {k_id} path: {' -> '.join(path_str)}")
            elif len(path_int) == 2 and path_int[0] == depot_int and path_int[1] == depot_int:
                depot_id = int_to_id[depot_int]
                print(f"Vehicle {k_id} path: {depot_id} -> {depot_id} (didn't use)")

Part 1: Data preparation and index mapping
Number of stations: 106
Number of stations with valid coordinates：106
Established station_id <-> index mapping
 total move_in = 150  total move_out = 150
Loaded all station coordinates
Distance matrix over integer indices saved to distance_matrix_int_km.csv
Part2: computing heuristic best depot candidate...
Heuristic result: best depot candidate is 1019 (index: 50)
Data preparation finished. 106 valid stations in total.
VRP fitness function defined
--- Running GA: Depot=1019, K=1 vehichcles ---


/opt/anaconda3/envs/Xpress/lib/python3.12/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/opt/anaconda3/envs/Xpress/lib/python3.12/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


    K=1 finished. minmum cost: 199.94
--- Running GA: Depot=1019, K=2 vehichcles ---
    K=2 finished. minmum cost: 193.54
--- Running GA: Depot=1019, K=3 vehichcles ---
    K=3 finished. minmum cost: 222.65

 Optimization finished (total time: 274.20 s) 
Depot: 1019
Vehicles: 2
Minimum total cost: 193.54

 Final rebalancing routes
Vehicles 1 path: 1019 -> 1028 -> 256 -> 275 -> 261 -> 877 -> 1738 -> 1818 -> 1720 -> 248 -> 1823 -> 296 -> 1821 -> 1721 -> 1767 -> 1726 -> 1744 -> 354 -> 889 -> 1799 -> 1766 -> 260 -> 264 -> 249 -> 290 -> 1025 -> 1096 -> 343 -> 262 -> 1097 -> 1026 -> 285 -> 1730 -> 890 -> 246 -> 1728 -> 1800 -> 358 -> 870 -> 259 -> 1098 -> 1822 -> 1860 -> 1092 -> 359 -> 1051 -> 252 -> 189 -> 2268 -> 346 -> 1808 -> 1731 -> 1737 -> 1764 -> 251 -> 1050 -> 885 -> 345 -> 1102 -> 1727 -> 273 -> 183 -> 255 -> 1757 -> 1055 -> 881 -> 265 -> 1093 -> 225 -> 1743 -> 1038 -> 1052 -> 1798 -> 342 -> 247 -> 1756 -> 355 -> 356 -> 1090 -> 820 -> 1769 -> 284 -> 1824 -> 253 -> 344 -> 340 -> 174

#### As it is a heuristic algorithm, we selected the optimal result from ten times run：

调度中心 (Depot): 1019

调度车辆数量 (K): 3

最小总成本（minimum total cost）: 184.55

--- 最终调度路径 (原始 ID) ---

车辆 1 路径: 1019 -> 1019 (未启用/空载)

车辆 2 路径: 1019 -> 265 -> 1727 -> 354 -> 877 -> 1098 -> 289 -> 1821 -> 1092 -> 260 -> 359 -> 881 -> 870 -> 183 -> 344 -> 1813 -> 284 -> 1737 -> 261 -> 1726 -> 189 -> 1051 -> 346 -> 290 -> 1765 -> 352 -> 343 -> 1731 -> 275 -> 1823 -> 1822 -> 1747 -> 1025 -> 1799 -> 1766 -> 1859 -> 1860 -> 1807 -> 1738 -> 249 -> 340 -> 246 -> 251 -> 1096 -> 2268 -> 356 -> 1743 -> 1769 -> 285 -> 1090 -> 1768 -> 253 -> 255 -> 1017 -> 1824 -> 355 -> 1720 -> 1728 -> 1763 -> 889 -> 890 -> 256 -> 1748 -> 248 -> 1102 -> 1055 -> 876 -> 1757 -> 1019

车辆 3 路径: 1019 -> 1798 -> 885 -> 273 -> 225 -> 2259 -> 1767 -> 264 -> 259 -> 358 -> 1038 -> 820 -> 345 -> 296 -> 247 -> 1818 -> 349 -> 1730 -> 171 -> 1800 -> 1039 -> 1764 -> 1745 -> 879 -> 1097 -> 252 -> 1028 -> 1808 -> 1721 -> 1024 -> 342 -> 351 -> 1744 -> 1050 -> 1756 -> 1026 -> 1093 -> 262 -> 1052 -> 1019